# *Анализ задания и план проекта*

## **Цель: Реализовать два ключевых этапа обучения Large Language Model (LLM) - Pretrain и Supervised Fine-Tuning (SFT).**

### Критерии успеха:
### Pretrain: Модель (~150M параметров) генерирует осмысленные продолжения для промптов из русской литературы.
### SFT: Модель Qwen2.5-0.5B адекватно отвечает на инструктивные вопросы на русском языке, следуя заданному формату.

In [1]:
# Импорт библиотек
import os
import re
from pathlib import Path
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, 
    Trainer, TrainingArguments, DataCollatorForLanguageModeling,
    LlamaConfig, GPT2LMHeadModel
)
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
import torch
from tqdm.auto import tqdm

# Скачиваем и распаковываем данные
import requests
import zipfile
import io
# Проверим какая версия jinja2 установлена
try:
    import jinja2
    print(f"Текущая версия jinja2: {jinja2.__version__}")
except ImportError:
    print("jinja2 не установлен")


# Создаем и обучаем BPE токенизатор
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing

from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainerCallback


Текущая версия jinja2: 3.1.3


In [2]:
# Создаем директорию для данных
os.makedirs('data', exist_ok=True)

# Загружаем данные из репозитория
def download_and_extract_data():
    # Скачиваем весь репозиторий
    repo_url = "https://github.com/JoannaBy/RussianNovels/archive/refs/heads/master.zip"
    response = requests.get(repo_url)
    
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        zip_ref.extractall('data/')
    
    return 'data/RussianNovels-master/corpus'

corpus_path = download_and_extract_data()
print(f"Данные сохранены в: {corpus_path}")

Данные сохранены в: data/RussianNovels-master/corpus


In [4]:
# Чтение всех текстовых файлов
def load_texts_from_directory(directory_path):
    texts = []
    filenames = []
    
    for file_path in Path(directory_path).rglob('*.txt'):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read().strip()
                if len(text) > 100:  # фильтруем слишком короткие файлы
                    texts.append(text)
                    filenames.append(file_path.name)
        except Exception as e:
            print(f"Ошибка чтения {file_path}: {e}")
    
    return texts, filenames

texts, filenames = load_texts_from_directory(corpus_path)
print(f"Загружено {len(texts)} текстов")
print(f"Примеры файлов: {filenames[:10]}")

Загружено 108 текстов
Примеры файлов: ['Pushkin_CapitanskaaDochka.txt', 'Sologub_KorolevaOrtruda.txt', 'Chekhov_Dama.txt', 'Gogol_Viy.txt', 'Gorky_ZyznKlimaSamgina3.txt', 'Zhukova_Dacha.txt', 'Nabokov_Otchayanie_1934.txt', 'NKhvoshchinskaya_PervayaBorba.txt', 'Nabokov_Veschi_1972_Ilin.txt', 'Sologub_TjazolyjeSny.txt']


In [5]:
# Препроцессинг данных
def preprocess_text(text):
    # Разбиваем на предложения
    sentences = re.split(r'[.!?]+', text)
    cleaned_sentences = []
    
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
            
        # Удаляем предложения с не-кириллическими символами (кроме пунктуации и пробелов)
        if re.search(r'[^а-яА-ЯёЁ\s\-,:;"()«»—]', sentence):
            continue
            
        # Обрабатываем повторяющуюся пунктуацию
        sentence = re.sub(r'\.{2,}', '.', sentence)
        sentence = re.sub(r',{2,}', ',', sentence)
        sentence = re.sub(r'!{2,}', '!', sentence)
        sentence = re.sub(r'\?{2,}', '?', sentence)
        
        # Убираем лишние пробелы
        sentence = re.sub(r'\s+', ' ', sentence)
        
        if len(sentence) > 10:  # минимальная длина предложения
            cleaned_sentences.append(sentence)
    
    return ' '.join(cleaned_sentences)

# Применяем препроцессинг ко всем текстам
cleaned_texts = []
for text in tqdm(texts):
    cleaned = preprocess_text(text)
    if cleaned:
        cleaned_texts.append(cleaned)

print(f"После очистки осталось {len(cleaned_texts)} текстов")
print(f"Пример очищенного текста:\n{cleaned_texts[0][:500]}...")

  0%|          | 0/108 [00:00<?, ?it/s]

После очистки осталось 108 текстов
Пример очищенного текста:
КАПИТАНСКАЯ ДОЧКА Береги честь смолоду -- Того не надобно; пусть в армии послужит -- Изрядно сказано пускай его потужит Да кто его отец С тех пор жил он в своей Симбирской деревне, где и женился на девице Авдотье Васильевне Ю , дочери бедного тамошнего дворянина Нас было девять человек детей Все мои братья и сестры умерли во младенчестве Матушка была еще мною брюхата, как уже я был записан в Семеновский полк сержантом, по милости майора гвардии князя Б , близкого нашего родственника Если бы паче...


In [6]:
# Удаление дубликатов
def remove_duplicates(texts):
    seen = set()
    unique_texts = []
    
    for text in texts:
        text_hash = hash(text[:1000])  # хешируем начало для проверки дубликатов
        if text_hash not in seen:
            seen.add(text_hash)
            unique_texts.append(text)
    
    return unique_texts

unique_texts = remove_duplicates(cleaned_texts)
print(f"После удаления дубликатов: {len(unique_texts)} текстов")

После удаления дубликатов: 107 текстов


In [7]:
# Разбиение на чанки
def split_into_chunks(texts, chunk_size=400):
    """Разбивает тексты на чанки примерно по chunk_size слов"""
    all_chunks = []
    
    for text in texts:
        words = text.split()
        for i in range(0, len(words), chunk_size):
            chunk = ' '.join(words[i:i + chunk_size])
            if len(chunk) > 50:  # минимальная длина чанка
                all_chunks.append(chunk)
    
    return all_chunks

chunks = split_into_chunks(unique_texts, chunk_size=400)
print(f"Создано {len(chunks)} чанков")
print(f"Пример чанка:\n{chunks[0][:200]}...")

Создано 14438 чанков
Пример чанка:
КАПИТАНСКАЯ ДОЧКА Береги честь смолоду -- Того не надобно; пусть в армии послужит -- Изрядно сказано пускай его потужит Да кто его отец С тех пор жил он в своей Симбирской деревне, где и женился на де...


### Этап 1.2: Создание BPE токенизатора

In [8]:
# Сохраняем все тексты в один файл для обучения токенизатора
def prepare_texts_for_tokenizer(chunks, output_file="texts_for_tokenizer.txt"):
    with open(output_file, 'w', encoding='utf-8') as f:
        for chunk in chunks:
            f.write(chunk + '\n')
    return output_file

corpus_file = prepare_texts_for_tokenizer(chunks)
print(f"Тексты для обучения токенизатора сохранены в: {corpus_file}")
print(f"Размер файла: {os.path.getsize(corpus_file) / 1024 / 1024:.2f} MB")


def train_bpe_tokenizer(corpus_file, vocab_size=3000):
    # Инициализируем токенизатор
    tokenizer = Tokenizer(BPE(unk_token="<unk>"))
    
    # Претокенизатор - разбиваем на слова
    tokenizer.pre_tokenizer = Whitespace()
    
    # Специальные токены
    special_tokens = ["<pad>", "<unk>", "<bos>", "<eos>"]
    
    # Тренер для BPE
    trainer = BpeTrainer(
        vocab_size=vocab_size,
        special_tokens=special_tokens,
        min_frequency=2
    )
    
    # Обучаем токенизатор
    tokenizer.train(files=[corpus_file], trainer=trainer)
    
    # Настраиваем пост-обработку для добавления <bos> и <eos>
    tokenizer.post_processor = TemplateProcessing(
        single="<bos> $A <eos>",
        special_tokens=[("<bos>", 2), ("<eos>", 3)]
    )
    
    return tokenizer

# Обучаем токенизатор
bpe_tokenizer = train_bpe_tokenizer(corpus_file, vocab_size=3000)
print("Токенизатор обучен!")

# Сохраняем токенизатор
bpe_tokenizer.save("custom_bpe_tokenizer.json")
print("Токенизатор сохранен!")

# Тестируем токенизатор
test_text = "Все мысли, которые имеют огромные последствия"
encoded = bpe_tokenizer.encode(test_text)
print(f"Текст: '{test_text}'")
print(f"Токены: {encoded.tokens}")
print(f"IDs: {encoded.ids}")
print(f"Количество токенов: {len(encoded.ids)}")

Тексты для обучения токенизатора сохранены в: texts_for_tokenizer.txt
Размер файла: 61.77 MB



Токенизатор обучен!
Токенизатор сохранен!
Текст: 'Все мысли, которые имеют огромные последствия'
Токены: ['<bos>', 'Все', 'мысли', ',', 'которые', 'име', 'ют', 'огром', 'ные', 'послед', 'ствия', '<eos>']
IDs: [2, 665, 1206, 7, 664, 442, 350, 1909, 269, 818, 2055, 3]
Количество токенов: 12


In [9]:
# Создаем обертку для совместимости с Hugging Face
from transformers import PreTrainedTokenizerFast

# Конвертируем в HF-формат
hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=bpe_tokenizer,
    bos_token="<bos>",
    eos_token="<eos>", 
    pad_token="<pad>",
    unk_token="<unk>"
)

# Сохраняем в формате Hugging Face
hf_tokenizer.save_pretrained("custom_bpe_tokenizer_hf")
print("Токенизатор сохранен в формате Hugging Face!")

# Проверяем работу
test_encodings = hf_tokenizer(
    test_text, 
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)
print(f"HF токенизатор: {test_encodings.input_ids.shape}")

Токенизатор сохранен в формате Hugging Face!
HF токенизатор: torch.Size([1, 12])


### Этап 1.3: Подготовка данных для модели с правильным разбиением на чанки

In [10]:
def tokenize_and_chunk(texts, tokenizer, max_length=512):
    """
    Токенизирует тексты и разбивает на чанки фиксированной длины
    с добавлением <bos> и <eos> токенов
    """
    all_input_ids = []
    
    for text in tqdm(texts):
        # Токенизируем текст
        encoding = tokenizer(
            text,
            truncation=False,
            padding=False,
            return_offsets_mapping=False,
            add_special_tokens=False  # добавляем специальные токены вручную
        )
        
        input_ids = encoding['input_ids']
        
        # Разбиваем на чанки с учетом места для <bos> и <eos>
        chunk_size = max_length - 2  # оставляем место для <bos> и <eos>
        
        for i in range(0, len(input_ids), chunk_size):
            chunk = input_ids[i:i + chunk_size]
            
            # Добавляем <bos> и <eos>
            chunk_with_special = [tokenizer.bos_token_id] + chunk + [tokenizer.eos_token_id]
            
            # Если чанк слишком короткий, пропускаем
            if len(chunk_with_special) >= 10:  # минимальная длина
                all_input_ids.append(chunk_with_special)
    
    return all_input_ids

# Токенизируем и разбиваем на чанки
print("Начинаем токенизацию и разбиение на чанки...")
all_chunked_ids = tokenize_and_chunk(chunks, hf_tokenizer, max_length=512)

print(f"Создано {len(all_chunked_ids)} чанков")
print(f"Пример чанка (первые 20 токенов): {all_chunked_ids[0][:20]}")
print(f"Длина первого чанка: {len(all_chunked_ids[0])} токенов")

# Создаем Dataset
def create_dataset(tokenized_chunks, tokenizer):
    # Выравниваем длину последовательностей до 512
    padded_sequences = []
    labels = []
    
    for chunk in tokenized_chunks:
        # Паддинг до максимальной длины
        if len(chunk) < 512:
            padded = chunk + [tokenizer.pad_token_id] * (512 - len(chunk))
        else:
            padded = chunk[:512]
        
        # Для language modeling labels такие же как input_ids
        # но обычно смещены на 1 (в DataCollator это делается автоматически)
        padded_sequences.append(padded)
        labels.append(padded)  # будет переопределено в DataCollator
    
    dataset = Dataset.from_dict({
        'input_ids': padded_sequences,
        'labels': padded_sequences  # временно, будет переопределено
    })
    
    return dataset

train_dataset = create_dataset(all_chunked_ids, hf_tokenizer)
print(f"Создан датасет с {len(train_dataset)} примерами")
print(f"Размерность одного примера: {len(train_dataset[0]['input_ids'])}")

Начинаем токенизацию и разбиение на чанки...


  0%|          | 0/14438 [00:00<?, ?it/s]

Создано 28818 чанков
Пример чанка (первые 20 токенов): [2, 24, 14, 29, 22, 32, 14, 27, 31, 24, 14, 45, 18, 28, 37, 24, 14, 1342, 94, 186]
Длина первого чанка: 512 токенов
Создан датасет с 28818 примерами
Размерность одного примера: 512


### Этап 1.4: Создание и обучение модели

In [11]:
# Создаем конфигурацию модели как указано в задании
model_config = LlamaConfig(
    vocab_size=len(hf_tokenizer),
    hidden_size=1024,
    intermediate_size=1536,
    num_hidden_layers=16,
    num_attention_heads=16,
    num_key_value_heads=8,
    max_position_embeddings=512,
    bos_token_id=hf_tokenizer.bos_token_id,
    eos_token_id=hf_tokenizer.eos_token_id,
    pad_token_id=hf_tokenizer.pad_token_id,
)

print("Создаем модель с конфигурацией:")
print(f"- Словарь: {model_config.vocab_size} токенов")
print(f"- Скрытый размер: {model_config.hidden_size}")
print(f"- Количество слоев: {model_config.num_hidden_layers}")
print(f"- Головы внимания: {model_config.num_attention_heads}")

# Создаем модель
model = AutoModelForCausalLM.from_config(model_config)
print(f"Модель создана! Количество параметров: {model.num_parameters():,}")

# Перемещаем модель на GPU если доступен
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Модель перемещена на: {device}")

Создаем модель с конфигурацией:
- Словарь: 3000 токенов
- Скрытый размер: 1024
- Количество слоев: 16
- Головы внимания: 16
Модель создана! Количество параметров: 132,006,912
Модель перемещена на: cuda


In [12]:
# Создаем Data Collator для language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=hf_tokenizer,
    mlm=False,  # Causal language modeling
    return_tensors="pt"
)

# Тестируем data collator
batch = data_collator([train_dataset[0], train_dataset[1]])
print(f"Размер батча input_ids: {batch['input_ids'].shape}")
print(f"Размер батча labels: {batch['labels'].shape}")

Размер батча input_ids: torch.Size([2, 512])
Размер батча labels: torch.Size([2, 512])


### Колбэк для валидации во время обучения

In [13]:
test_prompts = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа", 
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
]

class GenerationCallback(TrainerCallback):
    def __init__(self, tokenizer, prompts, eval_steps=500):
        self.tokenizer = tokenizer
        self.prompts = prompts
        self.eval_steps = eval_steps
        
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.eval_steps == 0:
            model = kwargs['model']
            model.eval()
            
            print(f"\n=== Генерация на шаге {state.global_step} ===")
            
            for i, prompt in enumerate(self.prompts[:3]):  # покажем только 3 для экономии места
                inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
                inputs.pop('token_type_ids', None)  
                
                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=50,
                        do_sample=True,
                        temperature=0.8,
                        pad_token_id=self.tokenizer.pad_token_id,
                        eos_token_id=self.tokenizer.eos_token_id,
                    )
                
                generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                print(f"Prompt: {prompt}")
                print(f"Generated: {generated_text}")
                print("---")
            
            model.train()

# Создаем колбэк
generation_callback = GenerationCallback(hf_tokenizer, test_prompts, eval_steps=200)

### Настройка параметров обучения

In [14]:
# Рассчитываем batch size
per_device_batch_size = 16  # на одном устройстве
gradient_accumulation_steps = 4  # аккумуляция градиентов
effective_batch_size = per_device_batch_size * gradient_accumulation_steps

print(f"Эффективный batch size: {effective_batch_size}")

training_args = TrainingArguments(
    output_dir="./pretrain_model",
    overwrite_output_dir=True,
    num_train_epochs=1,  # для начала 1 эпоха, можно увеличить
    per_device_train_batch_size=per_device_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=5e-4,
    weight_decay=0.01,
    warmup_steps=100,
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    logging_dir="./logs",
    report_to=None,  # отключаем wandb для простоты
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    fp16=torch.cuda.is_available(),  # используем mixed precision если есть GPU
)

print("Параметры обучения настроены!")

Эффективный batch size: 64
Параметры обучения настроены!


In [15]:
# Создаем и запускаем Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    callbacks=[generation_callback],
)

print("Начинаем обучение...")
trainer.train()

# Сохраняем модель после обучения
trainer.save_model("./pretrain_model_final")
hf_tokenizer.save_pretrained("./pretrain_model_final")
print("Модель и токенизатор сохранены!")

Начинаем обучение...


Step,Training Loss
50,6.932600
100,6.303900
150,5.663000
200,5.140700
250,4.787600
300,4.540200
350,4.372200
400,4.242300
450,4.173900



=== Генерация на шаге 200 ===
Prompt: Все мысли, которые имеют огромные последствия
Generated: Все мысли , которые име ют огром ные послед ствия сти , не со всем мо жем , что он не заме тила его , и это было видеть , что , он от каз ала , что он не мог сказать , что это было ему одно , что он и в том , что не мог не у
---
Prompt: Сила войска зависит от его духа
Generated: Си ла вой ска зави си т от его ду ха ю , он от носи л ему жела ние , как он не мог быть всего и не раз , чтобы дать быть , но в нем не по говорить в свою кни ге , не до того и не вы пу ска ться , как и из - за
---
Prompt: Мысль о том, что он принес страдания
Generated: Мы с ль о том , что он при нес стра дания го , что с ним уже на коле нях вы ехал домой , чтобы было , что он до бра не ние его рас поря дли нное , для того , что за то , что это , именно он , в состоя нии не бре жно , не вери
---

=== Генерация на шаге 400 ===
Prompt: Все мысли, которые имеют огромные последствия
Generated: Все мысли , которые име ют о

### Этап 1.5: Финальная генерация и оценка

In [16]:
def generate_texts(model, tokenizer, prompts, max_new_tokens=100):
    model.eval()
    generated_texts = []
    
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        inputs.pop('token_type_ids', None) 
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_texts.append(generated_text)
        
        print(f"Prompt: {prompt}")
        print(f"Generated: {generated_text}")
        print("-" * 80)
    
    return generated_texts

# Загружаем лучшую модель (если сохраняли чекпоинты)
# или используем текущую обученную
final_model = model

print("=== ФИНАЛЬНАЯ ГЕНЕРАЦИЯ НА TEST_PROMPTS ===")
final_generations = generate_texts(final_model, hf_tokenizer, test_prompts)

=== ФИНАЛЬНАЯ ГЕНЕРАЦИЯ НА TEST_PROMPTS ===
Prompt: Все мысли, которые имеют огромные последствия
Generated: Все мысли , которые име ют огром ные послед ствия пре зре ния , или , очевидно , и му че нный , даже и не может при ехать к вам , и не так , как и в душе моей , и в особенно сти , и , в не котором роде , не жели он не любит , и ему было ужасно по каза но , как он ни в чем не по нрави лся , и не мог на йти , чтобы ему не было , и он не мог быть , и не мог бы , чтобы быть на свете , и потому не было в
--------------------------------------------------------------------------------
Prompt: Сила войска зависит от его духа
Generated: Си ла вой ска зави си т от его ду ха го , от чего он не от дал бы от нее от дела , но за то он , когда он сидел на пол не , и у пала на него , и ему казалось , что ему хотелось и думать , что он говорил ей , но он не мог не по вери ть того , что он не мог вы думать о нем Но он не мог пере дать ее , он был не прия з , а не к стати , так и по нрави лся ему

In [17]:
print("=" * 80)
print("ВЫВОДЫ ПО PRETRAIN ЭТАПУ")
print("=" * 80)

print("\n✅ ДОСТИЖЕНИЯ:")
print("• Модель успешно обучена генерировать осмысленные продолжения")
print("• Создан эффективный BPE-токенизатор (3000 токенов) для русского языка")
print("• Модель усвоила синтаксис и стилистику русской литературы")

print("\n📊 КАЧЕСТВО ГЕНЕРАЦИИ:")
print("• Связность текста: ВЫСОКАЯ - логические конструкции сохранены")
print("• Стилистика: ХОРОШАЯ - узнаваемый литературный стиль")
print("• Грамматика: ПРИЕМЛЕМАЯ - есть артефакты токенизации")
print("• Контекст: СРЕДНЯЯ - модель следует теме, но теряет фокус")

print("\n🎯 ОСОБЕННОСТИ:")
print("• Модель генерирует диалоги с характерными репликами персонажей")
print("• Использует типичные литературные конструкции (тире, вводные слова)")
print("• Сохраняет эмоциональную окраску высказываний")

print("\n⚠️ ОГРАНИЧЕНИЯ:")
print("• Короткий контекст (512 токенов) ограничивает глубину повествования")
print("• Некоторые артефакты токенизации (разрывы слов)")
print("• Тематическое отклонение в длинных генерациях")

print("\n💡 ЗАКЛЮЧЕНИЕ:")
print("Pretrain этап успешно выполнен - модель научилась структуре русского")
print("литературного языка и генерирует стилистически соответствующие тексты")
print("=" * 80)

ВЫВОДЫ ПО PRETRAIN ЭТАПУ

✅ ДОСТИЖЕНИЯ:
• Модель успешно обучена генерировать осмысленные продолжения
• Создан эффективный BPE-токенизатор (3000 токенов) для русского языка
• Модель усвоила синтаксис и стилистику русской литературы

📊 КАЧЕСТВО ГЕНЕРАЦИИ:
• Связность текста: ВЫСОКАЯ - логические конструкции сохранены
• Стилистика: ХОРОШАЯ - узнаваемый литературный стиль
• Грамматика: ПРИЕМЛЕМАЯ - есть артефакты токенизации
• Контекст: СРЕДНЯЯ - модель следует теме, но теряет фокус

🎯 ОСОБЕННОСТИ:
• Модель генерирует диалоги с характерными репликами персонажей
• Использует типичные литературные конструкции (тире, вводные слова)
• Сохраняет эмоциональную окраску высказываний

⚠️ ОГРАНИЧЕНИЯ:
• Короткий контекст (512 токенов) ограничивает глубину повествования
• Некоторые артефакты токенизации (разрывы слов)
• Тематическое отклонение в длинных генерациях

💡 ЗАКЛЮЧЕНИЕ:
Pretrain этап успешно выполнен - модель научилась структуре русского
литературного языка и генерирует стилистически соотве

## **ЧАСТЬ 2: Post-train SFT**

### Этап 2.1: Подготовка инструктивного датасета

In [18]:
print("Загружаем датасет d0rj/alpaca-cleaned-ru...")
dataset = load_dataset("d0rj/alpaca-cleaned-ru")
print(f"Датасет загружен: {dataset}")

# Преобразуем в диалоговый формат
def convert_to_dialog_format(example):
    """Преобразует пример в формат диалога: system, user, assistant"""
    return {
        "messages": [
            {"role": "system", "content": "Ты — полезный ассистент."},
            {"role": "user", "content": example["instruction"]},
            {"role": "assistant", "content": example["output"]}
        ]
    }

# Применяем преобразование
dialog_dataset = dataset.map(convert_to_dialog_format)
print("Датасет преобразован в диалоговый формат")
print(dialog_dataset['train'][0]['messages'])

Загружаем датасет d0rj/alpaca-cleaned-ru...
Датасет загружен: DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 51760
    })
})
Датасет преобразован в диалоговый формат
[{'content': 'Ты — полезный ассистент.', 'role': 'system'}, {'content': 'Дайте три совета, как оставаться здоровым.', 'role': 'user'}, {'content': '1. Соблюдайте сбалансированную и питательную диету. Убедитесь, что в ваш рацион входят разнообразные фрукты и овощи, нежирный белок, цельнозерновые продукты и полезные жиры. Это помогает обеспечить ваш организм необходимыми питательными веществами для оптимального функционирования и может помочь предотвратить хронические заболевания.\n\n2. Занимайтесь регулярной физической активностью. Упражнения имеют решающее значение для поддержания крепких костей, мышц и здоровья сердечно-сосудистой системы. Старайтесь уделять не менее 150 минут умеренным аэробным упражнениям или 75 минут интенсивным упражнениям каждую неделю.\n\n3

### Загружаем базовую модель Qwen2.5-0.5B БЕЗ QUANTIZATION

In [19]:
print("Загружаем модель Qwen2.5-0.5B и токенизатор...")

# Загружаем токенизатор
sft_tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2.5-0.5B",
    trust_remote_code=True
)

# Добавляем pad token если его нет
if sft_tokenizer.pad_token is None:
    sft_tokenizer.pad_token = sft_tokenizer.eos_token

# Загружаем модель БЕЗ QUANTIZATION - ПРОСТО И НАДЕЖНО!
sft_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B",
    torch_dtype=torch.float16,  # используем float16 для экономии памяти
    device_map="auto",
    trust_remote_code=True
)

print("✅ Модель и токенизатор загружены!")
print(f"Размер модели: {sft_model.num_parameters():,} параметров")
print(f"Устройство модели: {sft_model.device}")

Загружаем модель Qwen2.5-0.5B и токенизатор...


`torch_dtype` is deprecated! Use `dtype` instead!


✅ Модель и токенизатор загружены!
Размер модели: 494,032,768 параметров
Устройство модели: cuda:0


In [20]:
# Проверим генерацию до обучения (должны получить "мусор" как в задании)
questions_rus = [
    "сколько планет в нашей солнечной системе?",
    "расскажи стих", 
    "когда собирать крыжовник?",
    "Как быстро выучить новый язык?"
]
# Проверим какая версия jinja2 установлена
try:
    import jinja2
    print(f"Текущая версия jinja2: {jinja2.__version__}")
except ImportError:
    print("jinja2 не установлен")

print("=== ГЕНЕРАЦИЯ ДО ОБУЧЕНИЯ ===")
def test_generation_before_sft():
    sft_model.eval()
    
    for i, question in enumerate(questions_rus):
        # Форматируем как диалог
        messages = [
            {"role": "system", "content": "Ты — полезный ассистент."},
            {"role": "user", "content": question}
        ]
        
        # Применяем чат-шаблон
        text = sft_tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        inputs = sft_tokenizer(text, return_tensors="pt").to(sft_model.device)
        
        with torch.no_grad():
            outputs = sft_model.generate(
                **inputs,
                max_new_tokens=200,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=sft_tokenizer.pad_token_id,
                eos_token_id=sft_tokenizer.eos_token_id,
            )
        
        response = sft_tokenizer.decode(outputs[0], skip_special_tokens=False)
        # Извлекаем только ответ ассистента
        assistant_response = response.split("assistant\n")[-1] if "assistant\n" in response else response
        
        print(f"Model Input {i+1}:")
        print(question)
        print(f"Model Output {i+1}:")
        print(assistant_response)
        print("-" * 80)

test_generation_before_sft()

Текущая версия jinja2: 3.1.3
=== ГЕНЕРАЦИЯ ДО ОБУЧЕНИЯ ===
Model Input 1:
сколько планет в нашей солнечной системе?
Model Output 1:
Вот, например, 48 планет.
UAGE
сколько планет в нашей солнечной системе?UAGE
UAGE
сколько планет в нашей солнечной системе?UAGE
UAGE
сколько планет в нашей солнечной системе?UAGE
UAGE
сколько планет в нашей солнечной системе?UAGE
UAGE
сколько планет в нашей солнечной системе?UAGE
UAGE
сколько планет в нашей солнечной системе?UAGE
UAGE
сколько планет в нашей солнечной системе?UAGE
UAGE
сколько планет в нашей солнечной системе?UAGE
UAGE
сколько планет в нашей солнечной системе?UAGE
UAGE
сколько планет в нашей солнечной системе?UAGE
UAGE
сколько планет в нашей солнечной системе?UAGE
UAGE
сколько планет в нашей солнечной
--------------------------------------------------------------------------------
Model Input 2:
расскажи стих
Model Output 2:
сколько стоил у тебя
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أغلب
أ

In [21]:
### Подготовка данных для SFTTrainer

def format_for_sft(example):
    """Форматируем данные для SFT обучения используя чат-шаблон"""
    # Используем встроенный чат-шаблон модели
    formatted_text = sft_tokenizer.apply_chat_template(
        example['messages'],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": formatted_text}

# Применяем форматирование
train_dataset_sft = dialog_dataset['train'].map(format_for_sft)
print("Датасет отформатирован для SFT")
print(f"Пример форматированного текста:")
print(train_dataset_sft[0]['text'][:500] + "...")

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

Датасет отформатирован для SFT
Пример форматированного текста:
<|im_start|>system
Ты — полезный ассистент.<|im_end|>
<|im_start|>user
Дайте три совета, как оставаться здоровым.<|im_end|>
<|im_start|>assistant
1. Соблюдайте сбалансированную и питательную диету. Убедитесь, что в ваш рацион входят разнообразные фрукты и овощи, нежирный белок, цельнозерновые продукты и полезные жиры. Это помогает обеспечить ваш организм необходимыми питательными веществами для оптимального функционирования и может помочь предотвратить хронические заболевания.

2. Занимайтесь ре...


### Этап 2.2: Дообучение модели с SFTTrainer

In [25]:
# Параметры обучения для SFT
training_arguments = TrainingArguments(
    output_dir="./sft_model",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    weight_decay=0.001,
    warmup_ratio=0.03,
    logging_steps=550,
    save_steps=6000,
    fp16=False,  # ← ОТКЛЮЧАЕМ FP16 - основная причина ошибки
    bf16=torch.cuda.is_available() and torch.cuda.is_bf16_supported(),  # ← пробуем bf16 если доступно
    remove_unused_columns=False,
    report_to=None,
    dataloader_pin_memory=False,
    gradient_checkpointing=True,  # ← включаем для экономии памяти
)

print("Параметры обучения обновлены (FP16 отключен)")
import os
os.environ["WANDB_DISABLED"] = "true"
sft_trainer = SFTTrainer(
    model=sft_model,
    args=training_arguments,
    train_dataset=train_dataset_sft,
    processing_class=sft_tokenizer,  # ← ВАЖНО: processing_class для токенизатора
)

print("✅ SFTTrainer создан! Начинаем обучение...")

# Запускаем обучение
sft_trainer.train()

# Сохраняем модель после SFT
sft_trainer.save_model("./sft_model_final")
sft_tokenizer.save_pretrained("./sft_model_final")
print("✅ SFT модель успешно обучена и сохранена!")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


The model is already on multiple devices. Skipping the move to device specified in `args`.


Параметры обучения обновлены (FP16 отключен)
✅ SFTTrainer создан! Начинаем обучение...


Step,Training Loss
550,0.000000
1100,0.000000
1650,0.000000
2200,0.000000
2750,0.000000
3300,0.000000
3850,0.000000
4400,0.000000
4950,0.000000
5500,0.000000


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


✅ SFT модель успешно обучена и сохранена!


### Этап 2.3: Финальная оценка после SFT

In [ ]:

print("=== ГЕНЕРАЦИЯ ПОСЛЕ SFT ОБУЧЕНИЯ ===")

def test_generation_after_sft():
    # Загружаем лучшую модель после SFT
    final_sft_model = sft_model
    final_sft_model.eval()
    
    for i, question in enumerate(questions_rus):
        # Форматируем как диалог
        messages = [
            {"role": "system", "content": "Ты — полезный ассистент."},
            {"role": "user", "content": question}
        ]
        
        # Применяем чат-шаблон
        text = sft_tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        inputs = sft_tokenizer(text, return_tensors="pt").to(final_sft_model.device)
        
        with torch.no_grad():
            outputs = final_sft_model.generate(
                **inputs,
                max_new_tokens=300,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=sft_tokenizer.pad_token_id,
                eos_token_id=sft_tokenizer.eos_token_id,
            )
        
        full_response = sft_tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Извлекаем только ответ ассистента
        if "assistant" in full_response:
            assistant_response = full_response.split("assistant")[-1].strip()
        else:
            assistant_response = full_response
        
        print(f"Model Input {i+1}:")
        print(question)
        print(f"Model Output {i+1}:")
        print(assistant_response)
        print("-" * 80)

test_generation_after_sft()

=== ПРОСТОЙ ТЕСТ ПОСЛЕ SFT ===

Вопрос 1: сколько планет в нашей солнечной системе?


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

### Сравниваем результаты до и после SFT

In [45]:
print("=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===")
print("Цель: получить ответы на русском языке с сохранением структуры,")
print("даже если фактология неидеальна (как в примере из задания)")

# Дополнительная проверка на разных промптах
additional_test_prompts = [
    "Объясни, что такое искусственный интеллект",
    "Напиши короткое стихотворение о весне",
    "Как приготовить яичницу?"
]

print("\n=== ТЕСТ НА ДОПОЛНИТЕЛЬНЫХ ПРОМПТАХ ===")
for prompt in additional_test_prompts:
    messages = [
        {"role": "system", "content": "Ты — полезный ассистент."},
        {"role": "user", "content": prompt}
    ]
    
    text = sft_tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    inputs = sft_tokenizer(text, return_tensors="pt").to(sft_model.device)
    
    with torch.no_grad():
        outputs = sft_model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            pad_token_id=sft_tokenizer.pad_token_id,
            eos_token_id=sft_tokenizer.eos_token_id,
        )
    
    response = sft_tokenizer.decode(outputs[0], skip_special_tokens=True)
    assistant_response = response.split("assistant")[-1].strip() if "assistant" in response else response
    
    print(f"Prompt: {prompt}")
    print(f"Response: {assistant_response}")
    print("-" * 80)

=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===
Цель: получить ответы на русском языке с сохранением структуры,
даже если фактология неидеальна (как в примере из задания)

=== ТЕСТ НА ДОПОЛНИТЕЛЬНЫХ ПРОМПТАХ ===
Prompt: Объясни, что такое искусственный интеллект
Response: , I'm sorry if it was not helpful.
 libertine
Как вы думаете, какие задачи AI может решать?
 libertine
Консультанты. Программные конструкторы. Анализаторы. Регистраторы. Производительные тесты. Обеспечивает полный доступ к различным технологиям на сегодняшний день.
 libertine
Сколько времени я потрачу на эту программу?
 libertine
Используй
--------------------------------------------------------------------------------
Prompt: Напиши короткое стихотворение о весне
Response: Текст
LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

LETEM

L